In [2]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import json
from requests_html import HTMLSession
import nest_asyncio
from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = []


ticker = 'ba'
sector = get_sector(ticker)
stock_hist = get_df(ticker)

index_ticker = '^GSPC'


index_hist = get_df(index_ticker)

years = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014]
columns = ['date', 'press title', 'urls','1d change', 'index_prices','norm_prices']

for year in years:

    nest_asyncio.apply()

    s = HTMLSession()
    url=f'https://investors.boeing.com/investors/news/{year}/default.aspx'
    r = s.get(url)
    ht= r.text
    soup = BeautifulSoup(ht) 


    press_statements = soup.find_all('span', {"class": "ModuleHeadline"})
    dates = soup.find_all('span', {"class": "ModuleDate"})
    urls = soup.find_all('a', {"class": 'ModuleHeadlineLink'})
    
    urls = [x.get("href") for x in urls]
    contents = []
    for url in urls:
        
        content = get_content_header(url, 'meta', {'name': 'twitter:description'})

        contents.append(content)
    
    dates = [datetime.strptime(x.text, "%b %d, %Y").date() for x in dates]
    press_statements = [x.text for x in press_statements]
    pct_changes = [changes_from_press(stock_hist, x, 1) for x in dates]
    
    norm_prices = []
    index_prices = []

    for date, pct_change in zip(dates, pct_changes):
        if pct_change == None:
            norm_prices.append(None)
            index_prices.append(None)            
        else:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
            norm_prices.append(norm_price)
            index_prices.append(index_price)
    dates = [str(x) for x in dates]
    data.extend(list(map(list, zip(dates, press_statements, urls, contents, pct_changes, index_prices, norm_prices))))

    
save_data(data, ticker, sector)


In [ ]:
urls